In [5]:
import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import matplotlib.pyplot as plt
import seaborn as sns
from pretwita import PreTwITA as PTW
from wordcloud import WordCloud
import PIL.Image 

<h1>Sentiment Analysis</h1>

In [7]:
df=pd.read_json('data_collection/data/df_mentions.json', orient='records', lines=True)

dfM_tweet= df.drop_duplicates(subset=['conversation_index'])
dfM_tweet_it= dfM_tweet[dfM_tweet['language']=='it']
#RENDO UGUALI LE LISTE PER EVITARE I NAN 
dfM_tweet_it.reset_index(inplace=True, drop=True)  #converte da Int a RageIndex

tweet_list= dfM_tweet_it['tweet'].tolist()

labels_list=[]
scores_list=[]
tweets_list=[]

tokenizer = AutoTokenizer.from_pretrained("MilaNLProc/feel-it-italian-sentiment")
model = AutoModelForSequenceClassification.from_pretrained("MilaNLProc/feel-it-italian-sentiment")

for sentence in tweet_list:
  inputs= tokenizer(sentence, return_tensors='pt', padding=True)

  # Call the model and get the logits
  labels = torch.tensor([1]).unsqueeze(0)  # Batch size 1
  outputs = model(**inputs, labels=labels)
  loss, logits = outputs[:2]
  logits = logits.squeeze(0)

  # Extract probabilities
  proba = torch.nn.functional.softmax(logits, dim=0)

  # Unpack the tensor to obtain negative and positive probabilities
  negative, positive = proba

  print('==============')
  print(sentence+'\n')
  print(f"Probabilities: Negative {np.round(negative.item(),4)} - Positive {np.round(positive.item(),4)}")
  print('============== \n')

  if ((np.round(negative.item(),4)>(np.round(positive.item(),4)))):
    #tweets.append(['NEGATIVE', np.round(negative.item(),4), sentence])
    labels_list.append('NEGATIVE')
    scores_list.append((np.round(negative.item(),4)))
    tweets_list.append(sentence)
  else:
    #tweets.append(['POSITIVE', np.round(positive.item(),4), sentence])
    labels_list.append('POSITIVE')
    scores_list.append((np.round(positive.item(),4)))
    tweets_list.append(sentence)

Downloading:   0%|          | 0.00/414 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/847 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/794k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.68M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/299 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/443M [00:00<?, ?B/s]

.@paolomieli: "Preoccupazione nei virologi come Fabrizio Pregliasco dopo le immagini degli stadi pieni agli Europei e dei festeggiamenti per l'approdo dell'#Italia alla finale di Wembley"

Probabilities: Negative 0.9998 - Positive 0.0002

#omnibus Il prof. Fabrizio Pregliasco su variante Delta e vaccini: "I vaccinati che si infettano non sviluppano malattia grave. Vediamo una differenza tra GB e Russia" @preglias #omnibusla7 #2luglio  https://t.co/K4HJzqiO7O

Probabilities: Negative 0.9997 - Positive 0.0003

Il virologo Fabrizio Pregliasco (@preglias) esprime apprezzamento per i colleghi della sanità sammarinese con cui in passato ha anche collaborato. Fondamentale mantenere il tracciamento per evitare le recrudescenze del virus  https://t.co/bk4dZdGvfw

Probabilities: Negative 0.0007 - Positive 0.9993

Non si esclude che mini #lockdown e mini #zonerosse potrebbero essere “necessarie” anche in un contesto di Italia tutta #zonabianca: lo ha detto il virologo Fabrizio Pregliasco  https:/

KeyboardInterrupt: 

In [ ]:
#crea dataframe per scores e salva in csv

df_scores=pd.DataFrame(columns=['label','score','tweet'])

df_scores['label']=labels_list
df_scores['score']=scores_list
df_scores['tweet']= tweets_list
df_scores['date']=dfM_tweet_it['date']
df_scores['username']=dfM_tweet_it['username']
df_scores['screen_name']=dfM_tweet_it['screen_name']


df_scores.to_csv('dfMentions_It_Sentiment.csv', index=False)

df_scores.head()

<h1>Graphs</h1>

In [ ]:
df= pd.read_csv('dfMentions_it_sentiment.csv')

df=df.drop(columns=('Unnamed: 0'))
df['label'].replace({'NEGATIVE':0, 'POSITIVE':1},inplace=True)
df['date'] = pd.to_datetime(df['date'])
df_date= pd.crosstab(df['date'],df['label'])

df.index = list(df.index)

In [ ]:
df_negative=df[df['label']==0]
df_positive=df[df['label']==1]

#SCREEN_NAMES
df_sName_pregliasco=df[df['screen_name'] == 'preglias' ]
df_sName_gallo=df[df['screen_name'] == 'drrobertcgallo' ]
df_sName_gualtieri=df[df['screen_name']== 'gualtierieurope']
df_sName_crisanti=df[df['screen_name']== '']
df_sName_galli=df[df['screen_name']== 'massimogalli']
df_sName_burioni= df[df['screen_name'] == 'robertoburioni']
df_sName_capua= df[df['screen_name'] == 'ilariacapua']
df_sName_zangrillo=df[df['screen_name']== 'azangrillo']
df_sName_ricciardi=df[df['screen_name']== 'wricciardi']
df_sName_bassetti=df[df['screen_name']== 'matteobassetti']
df_sName_brusaferro=df[df['screen_name']== 's_brusaferro']
df_sName_lopalco=df[df['screen_name']== 'proflopalco']
df_sName_cartabellotta=df[df['screen_name']== 'cartabellotta']

#SCREEN_NAME_CROSSED
df_sName_pregliasco_cross= pd.crosstab(df_sName_pregliasco['date'],df_sName_pregliasco['label'])
df_sName_gallo_cross= pd.crosstab(df_sName_gallo['date'],df_sName_gallo['label'])
df_sName_crisanti_cross= pd.crosstab(df_sName_crisanti['date'],df_sName_crisanti['label'])
df_sName_galli_cross= pd.crosstab(df_sName_galli['date'],df_sName_galli['label'])
df_sName_gualtieri_cross= pd.crosstab(df_sName_gualtieri['date'],df_sName_gualtieri['label'])
df_sName_burioni_cross= pd.crosstab(df_sName_burioni['date'],df_sName_burioni['label'])
df_sName_capua_cross= pd.crosstab(df_sName_capua['date'],df_sName_capua['label'])
df_sName_zangrillo_cross= pd.crosstab(df_sName_zangrillo['date'],df_sName_zangrillo['label'])
df_sName_ricciardi_cross= pd.crosstab(df_sName_ricciardi['date'],df_sName_ricciardi['label'])
df_sName_bassetti_cross= pd.crosstab(df_sName_bassetti['date'],df_sName_bassetti['label'])
df_sName_brusaferro_cross= pd.crosstab(df_sName_brusaferro['date'],df_sName_brusaferro['label'])
df_sName_lopalco_cross= pd.crosstab(df_sName_lopalco['date'],df_sName_lopalco['label'])
df_sName_cartabellotta_cross= pd.crosstab(df_sName_cartabellotta['date'],df_sName_cartabellotta['label'])

#SCREEN_NAMES positive
df_sName_pregliasco_P =df_positive[df_positive['screen_name'] == 'preglias' ]
df_sName_gallo_P =df_positive[df_positive['screen_name'] == 'drrobertcgallo' ]   #non sembra aver parlato di virologi
df_sName_gualtieri_P =df_positive[df_positive["screen_name"]== 'gualtierieurope'] #ha twittato solo 2 volte su virologi
df_sName_crisanti_P =df_positive[df_positive["screen_name"]== '']  #non sembra avere twitter
df_sName_galli_P =df_positive[df_positive["screen_name"]== 'massimogalli']  # non sembra aver parlato di virologi
df_sName_burioni_P = df_positive[df_positive['screen_name'] == 'robertoburioni']
df_sName_capua_P = df_positive[df_positive['screen_name'] == 'ilariacapua']
df_sName_zangrillo_P =df_positive[df_positive["screen_name"]== 'azangrillo']
df_sName_ricciardi_P =df_positive[df_positive["screen_name"]== 'wricciardi']
df_sName_bassetti_P =df_positive[df_positive["screen_name"]== 'matteobassetti'] #non sembra aver parlato di virologi
df_sName_brusaferro_P =df_positive[df_positive["screen_name"]== 's_brusaferro'] #non sembra aver parlato di virologi
df_sName_lopalco_P =df_positive[df_positive["screen_name"]== 'proflopalco']
df_sName_cartabellotta_P =df_positive[df_positive["screen_name"]== 'cartabellotta']

#SCREEN_NAME negative
df_sName_pregliasco_N =df_negative[df_negative['screen_name'] == 'preglias' ]
df_sName_gallo_N =df_negative[df_negative['screen_name'] == 'drrobertcgallo' ]   #non sembra aver parlato di virologi
df_sName_gualtieri_N =df_negative[df_negative["screen_name"]== 'gualtierieurope'] #ha twittato solo 2 volte su virologi
df_sName_crisanti_N =df_negative[df_negative["screen_name"]== '']  #non sembra avere twitter
df_sName_galli_N =df_negative[df_negative["screen_name"]== 'massimogalli']  # non sembra aver parlato di virologi
df_sName_burioni_N =df_negative[df_negative['screen_name'] == 'robertoburioni']
df_sName_capua_N =df_negative[df_negative['screen_name'] == 'ilariacapua']
df_sName_zangrillo_N =df_negative[df_negative["screen_name"]== 'azangrillo']
df_sName_ricciardi_N =df_negative[df_negative["screen_name"]== 'wricciardi']
df_sName_bassetti_N =df_negative[df_negative["screen_name"]== 'matteobassetti'] #non sembra aver parlato di virologi
df_sName_brusaferro_N =df_negative[df_negative["screen_name"]== 's_brusaferro'] #non sembra aver parlato di virologi
df_sName_lopalco_N =df_negative[df_negative["screen_name"]== 'proflopalco']
df_sName_cartabellotta_N =df_negative[df_negative["screen_name"]== 'cartabellotta']

#SCREEN_NAME_CROSSED_POSITIVE
df_sName_pregliasco_cross_P = pd.crosstab(df_sName_pregliasco_P['date'],df_sName_pregliasco_P['label'])
df_sName_gallo_cross_P = pd.crosstab(df_sName_gallo_P['date'],df_sName_gallo_P['label'])
df_sName_crisanti_cross_P = pd.crosstab(df_sName_crisanti_P['date'],df_sName_crisanti_P['label'])
df_sName_galli_cross_P = pd.crosstab(df_sName_galli_P['date'],df_sName_galli_P['label'])
df_sName_gualtieri_cross_P = pd.crosstab(df_sName_gualtieri_P['date'],df_sName_gualtieri_P['label'])
df_sName_burioni_cross_P = pd.crosstab(df_sName_burioni_P['date'],df_sName_burioni_P['label'])
df_sName_capua_cross_P = pd.crosstab(df_sName_capua_P['date'],df_sName_capua_P['label'])
df_sName_zangrillo_cross_P = pd.crosstab(df_sName_zangrillo_P['date'],df_sName_zangrillo_P['label'])
df_sName_ricciardi_cross_P = pd.crosstab(df_sName_ricciardi_P['date'],df_sName_ricciardi_P['label'])
df_sName_bassetti_cross_P = pd.crosstab(df_sName_bassetti_P['date'],df_sName_bassetti_P['label'])
df_sName_brusaferro_cross_P = pd.crosstab(df_sName_brusaferro_P['date'],df_sName_brusaferro_P['label'])
df_sName_lopalco_cross_P = pd.crosstab(df_sName_lopalco_P['date'],df_sName_lopalco_P['label'])
df_sName_cartabellotta_cross_P = pd.crosstab(df_sName_cartabellotta_P['date'],df_sName_cartabellotta_P['label'])

#SCREEN_NAME_CROSSED_NEGATIVE
df_sName_pregliasco_cross_N = pd.crosstab(df_sName_pregliasco_N['date'],df_sName_pregliasco_N['label'])
df_sName_gallo_cross_N = pd.crosstab(df_sName_gallo_N['date'],df_sName_gallo_N['label'])
df_sName_crisanti_cross_N = pd.crosstab(df_sName_crisanti_N['date'],df_sName_crisanti_N['label'])
df_sName_galli_cross_N = pd.crosstab(df_sName_galli_N['date'],df_sName_galli_N['label'])
df_sName_gualtieri_cross_N = pd.crosstab(df_sName_gualtieri_N['date'],df_sName_gualtieri_N['label'])
df_sName_burioni_cross_N = pd.crosstab(df_sName_burioni_N['date'],df_sName_burioni_N['label'])
df_sName_capua_cross_N = pd.crosstab(df_sName_capua_N['date'],df_sName_capua_N['label'])
df_sName_zangrillo_cross_N = pd.crosstab(df_sName_zangrillo_N['date'],df_sName_zangrillo_N['label'])
df_sName_ricciardi_cross_N = pd.crosstab(df_sName_ricciardi_N['date'],df_sName_ricciardi_N['label'])
df_sName_bassetti_cross_N = pd.crosstab(df_sName_bassetti_N['date'],df_sName_bassetti_N['label'])
df_sName_brusaferro_cross_N = pd.crosstab(df_sName_brusaferro_N['date'],df_sName_brusaferro_N['label'])
df_sName_lopalco_cross_N = pd.crosstab(df_sName_lopalco_N['date'],df_sName_lopalco_N['label'])
df_sName_cartabellotta_cross_N = pd.crosstab(df_sName_cartabellotta_N['date'],df_sName_cartabellotta_N['label'])


In [ ]:
#preparing for positve percentage
cartabellottaDF=df[df['screen_name']== 'cartabellotta']
pregliascoDF=df[df['screen_name'] == 'preglias' ]
galloDF=df[df['screen_name'] == 'drrobertcgallo' ]  
gualtieriDF=df[df['screen_name']== 'gualtierieurope'] 
crisantiDF=df[df['screen_name']== '']
galliDF=df[df['screen_name']== 'massimogalli']  
zangrilloDF=df[df['screen_name']== 'azangrillo']
ricciardiDF=df[df['screen_name']== 'wricciardi']
bassettiDF=df[df['screen_name']== 'matteobassetti'] 
brusaferroDF=df[df['screen_name']== 's_brusaferro'] 
lopalcoDF=df[df['screen_name']== 'proflopalco']
burioniDF = df[df['screen_name']=='robertoburioni']
capuaDF = df[df['screen_name']=='ilariacapua']

cartabellottaN = len(cartabellottaDF)
cartabellottaOnes = [1 for i in range(cartabellottaN)]

pregliascoN = len(pregliascoDF)
pregliascoOnes = [1 for i in range(pregliascoN)]

galloN = len(galloDF)
galloOnes = [1 for i in range(galloN)]

gualtieriN = len(gualtieriDF)
gualtieriOnes = [1 for i in range(gualtieriN)]

galliN = len(galliDF)
galliOnes = [1 for i in range(galliN)]

zangrilloN = len(zangrilloDF)
zangrilloOnes = [1 for i in range(zangrilloN)]

ricciardiN = len(ricciardiDF)
ricciardiOnes = [1 for i in range(ricciardiN)]

bassettiN = len(bassettiDF)
bassettiOnes = [1 for i in range(bassettiN)]

brusaferroN = len(brusaferroDF)
brusaferroOnes = [1 for i in range(brusaferroN)]

lopalcoN = len(lopalcoDF)
lopalcoOnes = [1 for i in range(lopalcoN)]

burioniN = len(burioniDF)
burioniOnes = [1 for i in range(burioniN)]

capuaN = len(capuaDF)
capuaOnes = [1 for i in range(capuaN)]

#CARTABELLOTTA
cartabellottaDF['ones']=cartabellottaOnes
groupedCartabellottaDF = cartabellottaDF.sort_values(by='date').groupby('date').sum()
division = groupedCartabellottaDF['label']/groupedCartabellottaDF['ones']
groupedCartabellottaDF['percentage'] = division
groupedCartabellottaDF
cartabellottaMean= str(int(division.mean()*100))

#PREGLIASCO
pregliascoDF['ones']=pregliascoOnes
groupedPregliascoDF = pregliascoDF.sort_values(by='date').groupby('date').sum()
division = groupedPregliascoDF['label']/groupedPregliascoDF['ones']
groupedPregliascoDF['percentage'] = division
groupedPregliascoDF
pregliascoMean= str(int(division.mean()*100))

#GUALTIERI
gualtieriDF['ones']=gualtieriOnes
groupedGualtieriDF = gualtieriDF.sort_values(by='date').groupby('date').sum()
division = groupedGualtieriDF['label']/groupedGualtieriDF['ones']
groupedGualtieriDF['percentage'] = division
groupedGualtieriDF
gualtieriMean= str(int(division.mean()*100))
#GALLI
galliDF['ones']=galliOnes
groupedGalliDF = galliDF.sort_values(by='date').groupby('date').sum()
division = groupedGalliDF['label']/groupedGalliDF['ones']
groupedGalliDF['percentage'] = division
groupedGalliDF
galliMean= str(int(division.mean()*100))
#ZANGRILLO
zangrilloDF['ones']=zangrilloOnes
groupedZangrilloDF = zangrilloDF.sort_values(by='date').groupby('date').sum()
division = groupedZangrilloDF['label']/groupedZangrilloDF['ones']
groupedZangrilloDF['percentage'] = division
groupedZangrilloDF
zangrilloMean= str(int(division.mean()*100))
#RICCIARDI
ricciardiDF['ones']=ricciardiOnes
groupedRicciardiDF = ricciardiDF.sort_values(by='date').groupby('date').sum()
division = groupedRicciardiDF['label']/groupedRicciardiDF['ones']
groupedRicciardiDF['percentage'] = division
groupedRicciardiDF
ricciardiMean= str(int(division.mean()*100))
#BASSETTI
bassettiDF['ones']=bassettiOnes
groupedBassettiDF = bassettiDF.sort_values(by='date').groupby('date').sum()
division = groupedBassettiDF['label']/groupedBassettiDF['ones']
groupedBassettiDF['percentage'] = division
groupedBassettiDF
bassettiMean= str(int(division.mean()*100))
#BRUSAFERRO
brusaferroDF['ones']=brusaferroOnes
groupedBrusaferroDF = brusaferroDF.sort_values(by='date').groupby('date').sum()
division = groupedBrusaferroDF['label']/groupedBrusaferroDF['ones']
groupedBrusaferroDF['percentage'] = division
groupedBrusaferroDF
brusaferroMean= str(int(division.mean()*100))
#LOPALCO
lopalcoDF['ones']=lopalcoOnes
groupedLopalcoDF = lopalcoDF.sort_values(by='date').groupby('date').sum()
division = groupedLopalcoDF['label']/groupedLopalcoDF['ones']
groupedLopalcoDF['percentage'] = division
groupedLopalcoDF
lopalcoMean= str(int(division.mean()*100))
#BURIONI
burioniDF['ones']=burioniOnes
groupedBurioniDF = burioniDF.sort_values(by='date').groupby('date').sum()
division = groupedBurioniDF['label']/groupedBurioniDF['ones']
groupedBurioniDF['percentage'] = division
groupedBurioniDF
burioniMean= str(int(division.mean()*100))
#CAPUA
capuaDF['ones']=capuaOnes
groupedCapuaDF = capuaDF.sort_values(by='date').groupby('date').sum()
division = groupedCapuaDF['label']/groupedCapuaDF['ones']
groupedCapuaDF['percentage'] = division
groupedCapuaDF
capuaMean= str(int(division.mean()*100))

In [ ]:
plt.figure(figsize=(10,8))
sns.set_context("paper", font_scale=2, rc={"font.size":20, "axes.titlesize":25, "axes.labelsize":20})
explode=(0.1,0)
g=df['label'].value_counts().plot(kind='pie',autopct="%1.0f%%",shadow=True,explode=explode, startangle=90, labels=['Negative', 'Positive'])

plt.ylabel('')

plt.savefig('SentimentAnalysis_pie.png',dpi=100)

In [ ]:
plt.figure(figsize=(15,8))
sns.set_context("paper", font_scale=2, rc={"font.size":17, "axes.titlesize":25, "axes.labelsize":20})
g=df['label'].value_counts().plot(kind='bar', color=['#2171b5','#f16913'])
plt.xticks([0,1],['Negative','Positive'],rotation=25)

#SCRIVE ALTEZZA BARRA SULLA CIMA
for p in g.patches:
    g.annotate(format(p.get_height(), '.0f'), (p.get_x() + p.get_width() / 2., p.get_height()), 
       ha = 'center', va = 'center', xytext = (0, 10), textcoords = 'offset points')

plt.savefig('SentimentAnalysis_histVal.png',dpi=100)

In [ ]:
#SCREEN_NAME POSITIVE (ricevuti)

sns.set_context("notebook", font_scale=1.5, rc={"font.size":10, "axes.titlesize":30, "axes.labelsize":25})
sns.set_style('darkgrid')
plt.plot(df_sName_pregliasco_cross_P, label='Pregliasco')
plt.plot(df_sName_galli_cross_P, label='Galli')
plt.plot(df_sName_gualtieri_cross_P, label='Gualtieri')
plt.plot(df_sName_burioni_cross_P, label='Burioni')
plt.plot(df_sName_capua_cross_P, label='Capua')
plt.plot(df_sName_zangrillo_cross_P, label='Zangrillo')
plt.plot(df_sName_ricciardi_cross_P, label='Ricciardi')
plt.plot(df_sName_bassetti_cross_P, label='Bassetti')
plt.plot(df_sName_brusaferro_cross_P, label='Brusaferro')
plt.plot(df_sName_lopalco_cross_P, label='Lopalco')
plt.plot(df_sName_cartabellotta_cross_P, label='Cartabellotta', color='yellow')

plt.gcf().set_size_inches(20, 10)

plt.ylabel('Frequency')

plt.legend(prop={'size': 20},facecolor='white')
plt.savefig('Sentiment_screenname_Positive_line.png',dpi=100)

In [ ]:
#SCREEN_NAME NEGATIVE (ricevuti)

sns.set_context("notebook", font_scale=1.5, rc={"font.size":10, "axes.titlesize":30, "axes.labelsize":25})
sns.set_style('darkgrid')
plt.plot(df_sName_pregliasco_cross_N, label='Pregliasco')
#plt.plot(df_sName_gallo_cross_N, label='drrobertcgallo')
#plt.plot(df_sName_crisanti_cross_N, label='')
plt.plot(df_sName_galli_cross_N, label='Galli')
plt.plot(df_sName_gualtieri_cross_N, label='Gualtieri')
plt.plot(df_sName_burioni_cross_N, label='Burioni')
plt.plot(df_sName_capua_cross_N, label='Capua')
plt.plot(df_sName_zangrillo_cross_N, label='Zangrillo')
plt.plot(df_sName_ricciardi_cross_N, label='Ricciardi')
plt.plot(df_sName_bassetti_cross_N, label='Bassetti')
plt.plot(df_sName_brusaferro_cross_N, label='Brusaferro')
plt.plot(df_sName_lopalco_cross_N, label='Lopalco')
plt.plot(df_sName_cartabellotta_cross_N, label='Cartabellotta', color='yellow')

plt.gcf().set_size_inches(20, 10)

plt.ylabel('Frequency')

plt.legend(prop={'size': 20},facecolor='white')
plt.savefig('Sentiment_screenname_Negative_line.png',dpi=100)

In [ ]:
#SCREEN_NAME POSITIVE PERCENTAGE (ricevuti)  [PT 1]

sns.set_context("paper", font_scale=1.5, rc={"font.size":10, "axes.titlesize":30, "axes.labelsize":25})
sns.set_style('darkgrid')
plt.plot(groupedBurioniDF['percentage'], label='Burioni'+"\n(mean: "+ burioniMean + "%)",color='tab:green')
plt.plot(groupedCapuaDF['percentage'], label='Capua '+"\n(mean: "+ capuaMean + "%)", color='tab:red')

plt.gcf().set_size_inches(20, 10)
 
plt.ylabel('Percentage frequency')

plt.gca().set_yticklabels(['{:.0f}%'.format(x*100) for x in plt.gca().get_yticks()])

plt.title("Positive sentiment distribution percentage per day\n (tweets received only)")
plt.legend(prop={'size': 20},bbox_to_anchor=(1.18, 1.01),loc='upper right')
plt.savefig('Sentiment_screenname_Positive_perc_line(PT1).png',bbox_inches='tight',dpi=100)
plt.show()

In [ ]:
#SCREEN_NAME POSITIVE PERCENTAGE (ricevuti)  [PT 2]

sns.set_context("paper", font_scale=1.5, rc={"font.size":10, "axes.titlesize":30, "axes.labelsize":25})
sns.set_style('darkgrid')
plt.plot(groupedBassettiDF['percentage'], label='Bassetti'+"\n(mean: "+ bassettiMean + "%)",color='forestgreen')
plt.plot(groupedCartabellottaDF['percentage'], label='Cartabellotta'+"\n(mean: "+ cartabellottaMean + "%)",color='coral')
plt.plot(groupedGalliDF['percentage'], label='Galli'+"\n(mean: "+ galliMean + "%)",color='tab:purple')
plt.plot(groupedGualtieriDF['percentage'], label='Gualtieri'+"\n(mean: "+ gualtieriMean + "%)",color='tab:brown')
plt.plot(groupedZangrilloDF['percentage'], label='Zangrillo'+"\n(mean: "+ zangrilloMean + "%)",color='tab:cyan')

plt.gcf().set_size_inches(20, 10)
 
plt.ylabel('Percentage frequency')

plt.gca().set_yticklabels(['{:.0f}%'.format(x*100) for x in plt.gca().get_yticks()])

plt.title("Positive sentiment distribution percentage per day\n (tweets received only)")
plt.legend(prop={'size': 20},bbox_to_anchor=(1.18, 1.01),loc='upper right')
plt.savefig('Sentiment_screenname_Positive_perc_line(PT2).png',bbox_inches='tight',dpi=100)
plt.show()

In [ ]:
#SCREEN_NAME POSITIVE PERCENTAGE (ricevuti)  [PT 3]

sns.set_context("paper", font_scale=1.5, rc={"font.size":10, "axes.titlesize":30, "axes.labelsize":25})
sns.set_style('darkgrid')
plt.plot(groupedBrusaferroDF['percentage'], label='Brusaferro'+"\n(mean: "+ brusaferroMean + "%)",color='tab:blue')
plt.plot(groupedLopalcoDF['percentage'], label='Lopalco'+"\n(mean: "+ lopalcoMean + "%)",color='tab:pink')
plt.plot(groupedPregliascoDF['percentage'], label='Pregliasco'+"\n(mean: "+ pregliascoMean + "%)",color='tab:gray')
plt.plot(groupedRicciardiDF['percentage'], label='Ricciardi'+"\n(mean: "+ ricciardiMean + "%)",color='tab:olive')

plt.gcf().set_size_inches(20, 10)
 
plt.ylabel('Percentage frequency')

plt.gca().set_yticklabels(['{:.0f}%'.format(x*100) for x in plt.gca().get_yticks()])

plt.title("Positive sentiment distribution percentage per day\n (tweets received only)")
plt.legend(prop={'size': 20},bbox_to_anchor=(1.18, 1.01),loc='upper right')
plt.savefig('Sentiment_screenname_Positive_perc_line(PT3).png',bbox_inches='tight',dpi=100)
plt.show()

<h1>Wordclouds</h1>

In [ ]:
list_positive = [tweet for tweet in df_positive['tweet']]
list_negative = [tweet for tweet in df_negative['tweet']]
list_all= [tweet for tweet in df['tweet']]

def Texts():
    text_positive=''
    text_negative=''
    text_all=''
    for tweet in list_positive:
        text_positive+=tweet
    for tweet in list_negative:
        text_negative+= tweet
    for tweet in list_all:
        text_all+=tweet
        
Texts()


In [ ]:
text=''
mask = np.array(PIL.Image.open('cloud.jpg'))

for tweet in list_positive:
    text+=tweet

ptw_positive=PTW(text)
clean_positive= ptw_positive.clean().text

my_list=clean_positive
def Mywordcloud():
    wordcloud= WordCloud(
    background_color='white',
    mask=mask,
    width=mask[0],
    height=mask[1],
    max_words=1000,
    max_font_size=60,
    scale=3,
    random_state=40,
    collocations=True,
    ).generate(str(my_list))
    
    fig=plt.figure(1,figsize=(20,20))
    plt.axis('off')

    plt.imshow(wordcloud)

Mywordcloud()
plt.savefig('Sentiment_Positive_Wordcloud.png',dpi=100)

In [ ]:
text=''

for tweet in list_negative:
    text+=tweet

ptw_negative=PTW(text)
clean_negative= ptw_negative.clean().text

my_list=clean_negative
def Mywordcloud():
    wordcloud= WordCloud(
    background_color='white',
    mask=mask,
    width=mask[0],
    height=mask[1],
    max_words=1000,
    max_font_size=60,
    scale=3,
    random_state=40,
    collocations=True,
    ).generate(str(my_list))
    
    fig=plt.figure(1,figsize=(20,20))
    plt.axis('off')
    plt.imshow(wordcloud)
    
Mywordcloud()
plt.savefig('Sentiment_Negative_Wordcloud.png',dpi=100)

In [ ]:
text=''

for tweet in list_all:
    text+=tweet

ptw_all=PTW(text)
clean_all= ptw_all.clean().text

my_list=clean_all
def Mywordcloud():
    wordcloud= WordCloud(
    background_color='white',
    mask=mask,
    width=mask[0],
    height=mask[1],
    max_words=1000,
    max_font_size=60,
    scale=3,
    random_state=40,
    collocations=True,
    ).generate(str(my_list))
    
    fig=plt.figure(1,figsize=(20,20))
    plt.axis('off')
    plt.imshow(wordcloud)


Mywordcloud()
plt.savefig('Sentiment_All_Wordcloud.png',dpi=100)